In [4]:
pip install botorch

     |████████████████████████████████| 357 kB 5.2 MB/s 
     |████████████████████████████████| 310 kB 58.8 MB/s 
     |████████████████████████████████| 713 kB 42.0 MB/s 
  Created wheel for gpytorch: filename=gpytorch-1.6.0-py2.py3-none-any.whl size=509889 sha256=0b13f503d1043ea3f8ca18dfdd51f3b258be6c7baf303d3e7b026f9858785492
  Stored in directory: /root/.cache/pip/wheels/66/b5/89/34c06ad393a6feb72b4cdde46d0f1c667f3e2632960f9df109
Successfully built gpytorch


In [37]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

import pandas as pd

train_data = pd.read_excel('data_sample.xlsx')
#print(train_data)
train_X = train_data.iloc[:,[0,1]].values
#print(train_X)
train_X = torch.tensor(train_X, dtype=torch.float32)
print(train_X)

train_Y = train_data.iloc[:,[2]].values
#print(train_Y)
train_Y = torch.tensor(train_Y, dtype=torch.float32)
print(train_Y)

gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)

tensor([[0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000]])
tensor([[0.0450],
        [0.0390],
        [0.0470],
        [0.0430],
        [0.0500],
        [0.0510],
        [0.0490],
        [0.0500],
        [0.0550],
        [0.0530],
        [0.0420],
        [0.0500],
        [0.0420],
        [0.0490],
        [0.0430],
        [0.0440],
        [0.0480],
        [0.0480],
        [0.0640],
        [0.0530],
        [0.0560

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

In [35]:
from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1)

In [40]:
from botorch.optim import optimize_acqf

#bounds = torch.stack([torch.zeros(2), torch.ones(2)])
bounds = torch.tensor([[0.005, 0.5],[0.0406, 2]])
#print(bounds)
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)
candidate

tensor([[0.0406, 1.5050]])